### Importing libraries

In [9]:
import pandas as pd
import numpy as np

### Importing the dataset

In [3]:
path ='/Users/maryrosejofelvillacampa/Desktop/datasets/astronomical_data.csv'
df = pd.read_csv(path)
df = pd.DataFrame(df)
df

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3042,0.0005,0.1542,16.6,0,Red,M
1,2600,0.0003,0.102,18.7,0,Red,M
2,2800,0.0002,,16.65,0,Red,M
3,1939,0.000138,0.103,20.06,0,Red,M
4,2840,,0.11,16.98,0,Red,M
...,...,...,...,...,...,...,...
234,38940,374830,1356,-9.93,5,Blue,O
235,30839,834042,1194,-10.63,5,Blue,O
236,8829,537493,1423,-10.73,5,White,A
237,9235,404940,1112,-11.23,5,White,A


### Data exploration

In [15]:
df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)", "Star type"]] = \
df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)", "Star type"]].apply(pd.to_numeric, errors='coerce')
df["Star color"] = pd.Categorical(df["Star color"])
df["Spectral Class"] = pd.Categorical(df["Spectral Class"])

df["Star color"] = df["Star color"].replace("\s", np.nan, regex=True)
df["Spectral Class"] = df["Spectral Class"].replace("\s", np.nan, regex=True)

shape = df.shape
print(f"There are {shape[0]} rows by {shape[1]} columns.")

dupes = df.duplicated().sum()
if dupes == 0:
    print("There are no duplicated rows in the dataset.")
else:
    print(f"There are {dupes} duplicated rows in the dataset")

print("The number of empty cells are listed below per feature.")
nulls = df.isnull().sum()
print(nulls)


There are 239 rows by 7 columns.
There are no duplicated rows in the dataset.
The number of empty cells are listed below per feature.
Temperature (K)           2
Luminosity(L/Lo)          6
Radius(R/Ro)              7
Absolute magnitude(Mv)    5
Star type                 0
Star color                6
Spectral Class            2
dtype: int64


## Data cleaning

### Temperature (K)

In [18]:
mean_temp = df["Temperature (K)"].mean()
df["Temperature (K)"] = df["Temperature (K)"].fillna(mean_temp)

null_temp = df["Temperature (K)"].isnull().sum()
if null_temp == 0:
    print("There are zero null values in the Temperature column")
else:
    print(f"There are {null_temp} empty cells in the column.")

There are zero null values in the Temperature column


### Luminosity (L/Lo)

In [19]:
mean_lum = df["Luminosity(L/Lo)"].mean()
df["Luminosity(L/Lo)"] = df["Luminosity(L/Lo)"].fillna(mean_lum)

null_lum = df["Luminosity(L/Lo)"].isnull().sum()
if null_lum == 0:
    print("There are zero null values in the Luminosity column")
else:
    print(f"There are {null_temp} empty cells in the column.")

There are zero null values in the Luminosity column


### Absolute magnitude (Mv)

In [20]:
mean_mv = df["Absolute magnitude(Mv)"].mean()
df["Absolute magnitude(Mv)"] = df["Absolute magnitude(Mv)"].fillna(mean_mv)

null_mv = df["Absolute magnitude(Mv)"].isnull().sum()
if null_mv == 0:
    print("There are zero null values in the Absolute magnitude column.")
else:
    print(f"There are {null_mv} empty cells in the column.")

There are zero null values in the Absolute magnitude column.
